In [1]:
import json
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
import os
os.chdir("/Users/saikrishnakambhampati/Downloads/Project/JSON/")
files=os.listdir("/Users/saikrishnakambhampati/Downloads/Project/JSON/")
data=[]
for file in files:    
    with open(file) as f:
        df = json.load(f)
        data.append(df)

In [3]:
print(data)

[{'ietf-mud:mud': {'mud-version': 1, 'mud-url': 'https://chromecastultra.com/chromecastultra', 'last-update': '2018-09-20T18:44:14.653+10:00', 'cache-validity': 100, 'is-supported': True, 'systeminfo': 'chromecastUltra', 'from-device-policy': {'access-lists': {'access-list': [{'name': 'from-ipv4-chromecastultra'}, {'name': 'from-ipv6-chromecastultra'}, {'name': 'from-ethernet-chromecastultra'}]}}, 'to-device-policy': {'access-lists': {'access-list': [{'name': 'to-ipv4-chromecastultra'}]}}}, 'ietf-access-control-list:access-lists': {'acl': [{'name': 'from-ipv4-chromecastultra', 'type': 'ipv4-acl-type', 'aces': {'ace': [{'name': 'from-ipv4-chromecastultra-0', 'matches': {'ipv4': {'protocol': 6, 'ietf-acldns:dst-dnsname': 'ad.doubleclick.net'}, 'tcp': {'destination-port': {'operator': 'eq', 'port': 443}, 'ietf-mud:direction-initiated': 'from-device'}}, 'actions': {'forwarding': 'accept'}}, {'name': 'from-ipv4-chromecastultra-1', 'matches': {'ietf-mud:mud': {'local-networks': [None]}, 'ipv

In [4]:
def getSourceDestination(data):
    if 'dnsname' in str(data):
        src_net_key = 'ietf-acldns:src-dnsname'
        dest_net_key = 'ietf-acldns:dst-dnsname'
    elif 'network' in str(data):
        src_net_key = 'source-ipv4-network'
        dest_net_key = 'destination-ipv4-network'
    else:
        src_net_key = '*'
        dest_net_key = '*'
    return data.get(src_net_key, '*'), data.get(dest_net_key, '*'), data.get('protocol', '*')
    

In [5]:
final_list = []
for i in range(len(data)):
    access_json=data[i].get('ietf-access-control-list:access-lists').get('acl')    
    for acl in access_json:
        aces = acl['aces']['ace']
        for ace in aces:
            final_row = {
            'sEth': '*',
            'dEth': '*',
            'typEth': '*',
            }
            if 'ethertype' in str(ace['matches']):
                final_row['sEth'] = ace['matches']['eth'].get('source-mac-address', '*')
                final_row['dEth'] = ace['matches']['eth'].get('destination-mac-address', '*')
                final_row['typEth'] = ace['matches']['eth'].get('ethertype', '*')
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            elif 'ipv6' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv6'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            else:
                final_row['Source'] = '*'
                final_row['Destination'] = '*'
                final_row['proto'] = '*'
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                if proto in [1,6,17] :
                    final_row['typEth']='0x800'
            if 'icmp' in str(ace['matches']):
                final_row['type'] = ace['matches']['icmp'].get('type', '*')
                final_row['code'] = ace['matches']['icmp'].get('code', '*')
            else:
                final_row['type'] = '*' 
                final_row['code'] = '*'   
            if 'source-port' in str(ace['matches']):
                final_row['sPort'] = ace['matches']['udp']['source-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['source-port'].get('port', '*')
            else:
                final_row['sPort'] = '*'
            if 'destination-port' in str(ace['matches']):
                final_row['dPort'] = ace['matches']['udp']['destination-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['destination-port'].get('port', '*')
            else:
                final_row['dPort'] = '*'
            final_row['priority'] = '*'
            final_row['action'] = 'forward' if ace['actions'].get('forwarding', '') == 'accept' else '*'
            if final_row['Source'] == '*' and final_row['Destination'] != '*':
                final_row['sEth'] = '9e:8d:de:80:29:28'  
            if final_row['Destination'] == '*' and final_row['Source'] != '*':
                final_row['dEth'] = '9e:8d:de:80:29:28'     
            final_list.append(final_row)

df = pd.DataFrame(final_list)
df


,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,0x800,*,ad.doubleclick.net,6,*,*,*,443,*,forward
1,9e:8d:de:80:29:28,*,*,*,224.0.0.251/32,2,*,*,*,*,*,forward
2,9e:8d:de:80:29:28,*,0x800,*,pagead2.googlesyndication.com,17,*,*,*,443,*,forward
3,9e:8d:de:80:29:28,*,0x800,*,www.googleapis.com,17,*,*,*,443,*,forward
4,9e:8d:de:80:29:28,*,0x800,*,ajax.googleapis.com,6,*,*,*,443,*,forward
5,*,*,0x800,*,*,17,*,*,*,53,*,forward
6,9e:8d:de:80:29:28,*,0x800,*,storage.googleapis.com,6,*,*,*,443,*,forward
7,9e:8d:de:80:29:28,*,0x800,*,googleads.g.doubleclick.net,6,*,*,*,443,*,forward
8,9e:8d:de:80:29:28,*,0x800,*,tools.google.com,17,*,*,*,443,*,forward
9,*,*,0x800,*,*,6,*,*,8009,*,*,forward


In [6]:
##df.to_csv("/Users/saikrishnakambhampati/Downloads/project/ACL_All_IOT_Devices.csv")